### cookbook

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = ChatOpenAI(temperature=.7)

In [4]:
chat([SystemMessage(content = "You are a helpful assistant that helps a user figure out what activity to do"),
      HumanMessage(content="I am bored and don't know what to do, I like outdoor activities")
])

AIMessage(content="How about going for a hike? It's a great way to enjoy the outdoors and get some exercise. You can find local trails or parks in your area and explore nature. Plus, it's a fantastic opportunity to disconnect from technology and enjoy some quiet time in nature.")